In [11]:
from __future__ import print_function
#from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import sys
from IPython.display import display, Image
from six.moves import cPickle as pickle
import tensorflow as tf
from tensorflow.python.ops import rnn_cell, rnn

In [101]:
from __future__ import division

In [5]:
#load data first
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS_time_serial.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    #train_dataset= save['train_dataset']
    #validate_dataset = save['validate_dataset']
    test_dataset = save['test_dataset']
    #test_old = save['v_t_dataset']
    train_old = save['t_v_dataset']
    del save

#train_time = train_dataset['time']
#train_data = train_dataset['data']
#train_label = train_dataset['label']
#validate_time = validate_dataset['time']
#validate_data = validate_dataset['data']
#validate_label = validate_dataset['label']
#test_time = test_dataset['time']
test_data = test_dataset['data']
test_label = test_dataset['label']
#test_old_data = test_old['data']
#test_old_label = test_old['label']
train_old_data = train_old['data']
train_old_label = train_old['label']

In [6]:
print(test_data.shape, test_label.shape)
print(train_old_data.shape, train_old_label.shape)

(29993, 8, 70) (29993, 8, 1)
(119966, 8, 70) (119966, 8, 1)


In [7]:
#dataset normalize
mean = train_old_data.mean(axis = (0,1))
std = train_old_data.std(axis = (0,1))
print(mean.shape, std.shape)
train_data_n = (train_old_data - mean)/std
#validate_data_n = (validate_data - mean)/std
test_data_n = (test_data - mean)/std

(70,) (70,)


In [23]:
train_label = train_old_label

# Simple LSTM regression Model 

In [22]:
a = test_label[:10, 7, :].shape
print(total_size)

119966


In [31]:
n_features = 70
n_steps = 8
n_labels = 1

n_hidden = 140
total_size = train_old_label.shape[0]

In [39]:


graph = tf.Graph()
with graph.as_default():
    inputs = tf.placeholder("float32", [None, n_steps, n_features])
    labels = tf.placeholder("float32", [None, n_labels])
    
    weights = {
        'hidden': tf.Variable(tf.random_normal([n_features, n_hidden])),
        'out': tf.Variable(tf.random_normal([n_hidden, n_labels]))
    }
    biases = {
        'hidden': tf.Variable(tf.random_normal([n_hidden])),
        'out': tf.Variable(tf.random_normal([n_labels]))
    }
    
    def RNN(x, w, b):
        # Prepare data shape to match `rnn` function requirements
        # Current data input shape: (batch_size, n_steps, n_input)
        # Required shape: 'n_steps' tensors list of shape (batch_size, n_hidden)
    
        # Permuting batch_size and n_steps
        x = tf.transpose(x, [1, 0, 2])
        # Reshaping to (n_steps*batch_size, n_input)
        x = tf.reshape(x, [-1, n_features])
        
        # Linear activation
        x = tf.matmul(x, w['hidden']) + b['hidden']
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_hidden)
        x = tf.split(0, n_steps, x)

        # Define a lstm cell with tensorflow
        lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)

        # Get lstm cell output
        outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

        # Linear activation, using rnn inner loop last output
        return tf.matmul(outputs[-1], w['out']) + b['out']
    
    pred = RNN(inputs, weights, biases)
    
    # Define loss and optimizer
    loss = tf.reduce_mean(tf.square(pred - labels))
    
    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.90, staircase=True)
    op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    MAE = tf.reduce_mean(tf.abs(pred - labels))

In [40]:
batch_size = 128
steps = 10000
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    test_feed_dict = {inputs:test_data_n, labels:test_label[:,-1,:]}
    for step in range(steps):
        off = step * batch_size % (total_size - batch_size)
        batch_data = train_data_n[off:off+batch_size, :, :]
        batch_label = train_label[off:off+batch_size, -1, :]
        feed_dict = {inputs:batch_data, labels:batch_label}
        l, _, r = session.run([loss, op, learning_rate], feed_dict=feed_dict)
        if step % 10 == 0:
            test_mae = MAE.eval(feed_dict=test_feed_dict)
            print('step: %d, LR = %f, min batch loss = %f, test MAE = %f' % (step, r, l, test_mae))
            #print('batch_data = %d %d %d, batch_label = %d, predicate = %f' % (batch_data[0, 0, 0],batch_data[0, 1, 0],batch_data[0, 2, 0], batch_label[0, 0], p))

Initialized
step: 0, LR = 0.050000, min batch loss = 133.201797, test MAE = 17.025288
step: 10, LR = 0.050000, min batch loss = 43.083408, test MAE = 4.369280
step: 20, LR = 0.050000, min batch loss = 6.099579, test MAE = 4.059394
step: 30, LR = 0.050000, min batch loss = 3.830898, test MAE = 3.841203
step: 40, LR = 0.050000, min batch loss = 17.280468, test MAE = 2.880110
step: 50, LR = 0.050000, min batch loss = 7.432853, test MAE = 3.305273
step: 60, LR = 0.050000, min batch loss = 20.624699, test MAE = 3.409530
step: 70, LR = 0.050000, min batch loss = 14.002827, test MAE = 2.497970
step: 80, LR = 0.050000, min batch loss = 7.022384, test MAE = 1.988173
step: 90, LR = 0.050000, min batch loss = 2.532738, test MAE = 1.972177
step: 100, LR = 0.050000, min batch loss = 7.286420, test MAE = 1.769380
step: 110, LR = 0.050000, min batch loss = 1.671923, test MAE = 1.687430
step: 120, LR = 0.050000, min batch loss = 2.427509, test MAE = 1.655931
step: 130, LR = 0.050000, min batch loss = 

# Simple LSTM classification Model

In [55]:
label1 = train_label.reshape(train_label.shape[0], train_label.shape[1])
label2 = test_label.reshape(test_label.shape[0], test_label.shape[1])

In [56]:
#Do classification later
from sklearn.preprocessing import Binarizer
pre = Binarizer(threshold = 1.01)
b_train_label = pre.transform(label1)
b_test_label = pre.transform(label2)

In [54]:
c_train_label = 1 - b_train_label
r, _ = (c_train_label == 1).nonzero()
r.shape, c_train_label.shape, c_train_label.reshape(c_train_label.shape[0]*c_train_label.shape[1], 1).shape

((14737,), (119966, 8), (959728, 1))

In [57]:
c_train_label = 1 - b_train_label
c_test_label = 1 - b_test_label

In [51]:
# change from Indice to Vector
''''''
def makeIndicatorVars(T):
    # Make sure T is two-dimensiona. Should be nSamples x 1.
    if T.ndim == 1:
        T = T.reshape((-1,1))    
    return (T == np.unique(T)).astype(int)

In [58]:
v_train_label_t = makeIndicatorVars(c_train_label.reshape(c_train_label.shape[0]*c_train_label.shape[1], 1))
v_test_label_t = makeIndicatorVars(c_test_label.reshape(c_test_label.shape[0]*c_test_label.shape[1], 1))

In [60]:
v_train_label = v_train_label_t.reshape(c_train_label.shape[0], c_train_label.shape[1], 2)
v_test_label = v_test_label_t.reshape(c_test_label.shape[0], c_test_label.shape[1], 2)


In [107]:
#need to balanced the dataset
from unbalanced_dataset.over_sampling import SMOTE
sm = SMOTE(ratio = 0.1, kind='regular')
train_data_n_resample, c_train_label_resample = sm.fit_transform(train_data_n, v_train_label)

ValueError: Found array with dim 3. Estimator expected <= 2.

In [91]:
#shuffle the data set
arr = np.arange(v_train_label.shape[0])
np.random.shuffle(arr)
train_data_n_s =  train_data_n[arr]
train_label_v_s = v_train_label[arr]

Simple LSTM Model

In [108]:
n_features = 70
n_steps = 8
n_labels = 2
n_hidden = 210
total_size = v_train_label.shape[0]

In [111]:


graph = tf.Graph()
with graph.as_default():
    inputs = tf.placeholder("float32", [None, n_steps, n_features])
    labels = tf.placeholder("float32", [None, n_labels])
    
    weights = {
        'hidden': tf.Variable(tf.random_normal([n_features, n_hidden])),
        'out': tf.Variable(tf.random_normal([n_hidden, n_labels]))
    }
    biases = {
        'hidden': tf.Variable(tf.random_normal([n_hidden])),
        'out': tf.Variable(tf.random_normal([n_labels]))
    }
    
    def acc(predict, label):
        #correct_prediction = tf.equal(predicted_label, tf_train_label)
        correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(label, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        predict_event = tf.reduce_sum(tf.argmax(predict, 1))
        label_event = tf.reduce_sum(tf.argmax(label, 1))
        true_positive = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 2), tf.int64))
        true_negative = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 0), tf.int64))
        false_positive = predict_event - true_positive 
        false_negative = label_event - true_positive
        return accuracy, false_positive, false_negative, true_positive, true_negative
    def ROC(FP, FN, TP, TN):
        TP_percent = TP / (TP + FN) 
        FP_percent = FP / (FP + TN) 
        return TP_percent, FP_percent
    
    def PRC(FP, FN, TP, TN):
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f_score = 2 * precision * recall / (precision + recall + 1)
        return precision, recall, f_score
    
    def RNN(x, w, b):
        # Prepare data shape to match `rnn` function requirements
        # Current data input shape: (batch_size, n_steps, n_input)
        # Required shape: 'n_steps' tensors list of shape (batch_size, n_hidden)
    
        # Permuting batch_size and n_steps
        x = tf.transpose(x, [1, 0, 2])
        # Reshaping to (n_steps*batch_size, n_input)
        x = tf.reshape(x, [-1, n_features])
        
        # Linear activation
        x = tf.matmul(x, w['hidden']) + b['hidden']
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_hidden)
        x = tf.split(0, n_steps, x)

        # Define a lstm cell with tensorflow
        lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)

        # Get lstm cell output
        outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

        # Linear activation, using rnn inner loop last output
        return tf.matmul(outputs[-1], w['out']) + b['out']
    
    pred = RNN(inputs, weights, biases)
    
    # Define loss and optimizer
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, labels)) # Softmax loss
    
    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.03
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.90 , staircase=True)
    op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    
    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    test_acc, FP, FN, TP, TN = acc(pred, labels)
    pre, rec, f_s = PRC(FP, FN, TP, TN)

In [112]:
batch_size = 128
steps = 10000
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    test_feed_dict = {inputs:test_data_n, labels:v_test_label[:,-1,:]}
    for step in range(steps):
        off = step * batch_size % (total_size - batch_size)
        batch_data = train_data_n_s[off:off+batch_size, :, :]
        batch_label = train_label_v_s[off:off+batch_size, -1, :]
        feed_dict = {inputs:batch_data, labels:batch_label}
        l, _, r = session.run([loss, op, learning_rate], feed_dict=feed_dict)
        if step % 10 == 0:
            acc = accuracy.eval(feed_dict=feed_dict)
            print('step: %d, LR = %f, min batch loss = %f, train acc = %f' % (step, r, l, acc))
        if step % 100 == 0:
            tp, fp, fn, tn, precision, recall, f_score= session.run([TP, FP, FN, TN, pre, rec, f_s], feed_dict=test_feed_dict)
            print("TP = %d, FP = %d, FN = %d, TN = %d" % (tp, fp, fn, tn))
            print("precision = %f, recall = %f, f_score = %f" % (precision, recall, f_score))
            #print('batch_data = %d %d %d, batch_label = %d, predicate = %f' % (batch_data[0, 0, 0],batch_data[0, 1, 0],batch_data[0, 2, 0], batch_label[0, 0], p))

Initialized
step: 0, LR = 0.030000, min batch loss = 3.624950, train acc = 0.976562
TP = 0, FP = 125, FN = 472, TN = 29396
precision = 0.000000, recall = 0.000000, f_score = 0.000000
step: 10, LR = 0.030000, min batch loss = 0.218847, train acc = 0.992188
step: 20, LR = 0.030000, min batch loss = 0.211309, train acc = 0.968750
step: 30, LR = 0.030000, min batch loss = 0.006116, train acc = 1.000000
step: 40, LR = 0.030000, min batch loss = 0.208186, train acc = 0.992188
step: 50, LR = 0.030000, min batch loss = 0.010709, train acc = 1.000000
step: 60, LR = 0.030000, min batch loss = 0.044564, train acc = 1.000000
step: 70, LR = 0.030000, min batch loss = 0.049560, train acc = 0.992188
step: 80, LR = 0.030000, min batch loss = 0.006031, train acc = 1.000000
step: 90, LR = 0.030000, min batch loss = 0.337211, train acc = 0.984375
step: 100, LR = 0.030000, min batch loss = 0.025463, train acc = 0.992188
TP = 49, FP = 91, FN = 423, TN = 29430
precision = 0.350000, recall = 0.103814, f_scor